# Imports

In [1]:
import os, sys, time, glob, random, argparse
import wandb
import numpy as np
from copy import deepcopy
import torch
import torch.nn as nn
import time
import tqdm
import scipy.stats as stats
import matplotlib.pyplot as plt
import pickle
import pandas as pd

# XAutoDL 
from xautodl.config_utils import load_config, dict2config, configure2str
from xautodl.datasets import get_datasets, get_nas_search_loaders
from xautodl.procedures import (
    prepare_seed,
    prepare_logger,
    save_checkpoint,
    copy_checkpoint,
    get_optim_scheduler,
)
from xautodl.utils import get_model_infos, obtain_accuracy
from xautodl.log_utils import AverageMeter, time_string, convert_secs2time
from xautodl.models import get_search_spaces

# API
from nats_bench import create

# custom modules
from custom.tss_model import TinyNetwork
from xautodl.models.cell_searchs.genotypes import Structure
from ZeroShotProxy import *
from tss_utils import compute_vkdnw, plot_stats, get_stats, get_metrics, analyze_results, generate_accs, get_results_from_api, get_scores

# All

In [2]:
# scp 'tyblondr@login.rci.cvut.cz:~/PycharmProjects/VKDNW/NB201/*_log.csv' data_0811/

target = 'val_accs'
dataset = 'ImageNet16-120'
compute_graf = False
zero_cost_score_list = ['vkdnw', 'vkdnw_dim', 'vkdnw_chisquare', 'az_nas', 'jacov','gradsign', 'zico', 'zen', 'grad_norm', 'naswot', 'synflow', 'snip', 'grasp', 'te_nas', 'flops', 'trainability', 'expressivity', 'progressivity']
compute_graf_str = 'filtered' if compute_graf else 'unfiltered'

In [3]:
run_dict = {
    'cifar10': {
        # same images
        #'nazderaze/VKDNW/n6n44keg': 1,
        #'nazderaze/VKDNW/6c1pv095': 2,
        #'nazderaze/VKDNW/45us11be': 3,
        #'nazderaze/VKDNW/rwp2qvkw': 4,
        #'nazderaze/VKDNW/6bsssh4z': 5,
        
        # different images
        #'nazderaze/VKDNW/7a8jm975' : -1 # testing (successfull)
        #'nazderaze/VKDNW/cixw6r9b': 1,
        #'nazderaze/VKDNW/1nikalf3': 2,
        #'nazderaze/VKDNW/r5l3qg12': 3,
        #'nazderaze/VKDNW/fq581h0d': 4,
        #'nazderaze/VKDNW/0yfzaxra': 5,
        
        # entropy
        #'nazderaze/VKDNW/87vy5rby': 1,
        #'nazderaze/VKDNW/8sd2hdws': 5,
        #'nazderaze/VKDNW/bh09isa0': 3,
        #'nazderaze/VKDNW/ei31s875': 4,
        #'nazderaze/VKDNW/koxirqt3': 2,
        
        # entropy all
        #'nazderaze/VKDNW/6lcy0jqp': 1,
        #'nazderaze/VKDNW/orl2me1l': 2,
        #'nazderaze/VKDNW/y5n4lnsn': 3,
        #'...': 4,
        #'nazderaze/VKDNW/7gkdl3gl': 5,
        
        # final
        'nazderaze/VKDNW/k7llaf2s': 1,
        'nazderaze/VKDNW/knu2gv65': 2,
        'nazderaze/VKDNW/fnfbpspk': 3,
        'nazderaze/VKDNW/w4b39d0d': 4,
        'nazderaze/VKDNW/vbr5n8iy': 5
        
    },
    'cifar100': {
        # same images
        #'nazderaze/VKDNW/qmq5vp3k': 1,
        #'nazderaze/VKDNW/31lrq6p7': 2,
        #'nazderaze/VKDNW/3qazc6po': 3,
        #'nazderaze/VKDNW/424twoyv': 4,
        #'nazderaze/VKDNW/783h4opf': 5,
        
        # different images
        #'nazderaze/VKDNW/n2m8i53l': 1,
        #'nazderaze/VKDNW/48xbnfdg': 2,
        #'nazderaze/VKDNW/f0czkx5u': 3,
        #'nazderaze/VKDNW/ldv3b1bh': 4,
        #'nazderaze/VKDNW/qwae4nqx': 5,
        
        # entropy
        #'nazderaze/VKDNW/lfnin2ui': 1,
        #'nazderaze/VKDNW/1cs8orlm': 4,
        #'nazderaze/VKDNW/4tep5lv0': 5,
        #'nazderaze/VKDNW/lqka0nwl': 2,
        #'nazderaze/VKDNW/mqrfyv7t': 3,
        
        # entropy all
        #'...': 1,
        #'...': 2,
        #'...': 3,
        #'...': 4,
        #'...': 5,
        
        # final
        'nazderaze/VKDNW/v2b816ul': 1,
        'nazderaze/VKDNW/velwxtxd': 2,
        'nazderaze/VKDNW/f3ljf5jf': 3,
        'nazderaze/VKDNW/e3w3dkv7': 4,
        'nazderaze/VKDNW/9ibvj04q': 5,
    },
    'ImageNet16-120': {
        # same images
        #'nazderaze/VKDNW/ftg0tdsa': 1,
        #'nazderaze/VKDNW/vqf1ey6x': 2,
        #'nazderaze/VKDNW/v0a0m67q': 3,
        #'nazderaze/VKDNW/uiv37u18': 4,
        #'nazderaze/VKDNW/c1338vfg': 5,
        
        # different images
        #'nazderaze/VKDNW/55f1omxn': 1,
        #'nazderaze/VKDNW/amdcxrz7': 2,
        #'nazderaze/VKDNW/sl0rjhwh': 3,
        #'nazderaze/VKDNW/z2ph6iav': 4,
        #'nazderaze/VKDNW/ol9rwkeo': 5,
        
        # entropy
        # 'nazderaze/VKDNW/hwtw58ot': -1 # test
        #'nazderaze/VKDNW/u214u8p1': 4,
        #'nazderaze/VKDNW/bycu6ed3': 5,
        #'nazderaze/VKDNW/2ztcsos6': 3,
        #'nazderaze/VKDNW/kptcxbi2': 2,
        #'nazderaze/VKDNW/f9n7j83e': 1,
        
        #'nazderaze/VKDNW/hrrxcexh': 1,
        #'nazderaze/VKDNW/jwhy017w': 2,
        #'nazderaze/VKDNW/82rycftz': 3,
        #'nazderaze/VKDNW/bw6b93p9': 4,
        #'nazderaze/VKDNW/ca423k7t': 5,
        
        # final
        'nazderaze/VKDNW/ss2kwvpp': 1,
        'nazderaze/VKDNW/es9t2696': 2,
        'nazderaze/VKDNW/m1dusbs9': 3,
        'nazderaze/VKDNW/kgmcpu2y': 4,
        'nazderaze/VKDNW/cpbmsvuw': 5,
        
    }
}

In [4]:
api_nats = create('/mnt/personal/tyblondr/NATS-tss-v1_0-3ffb9-simple/', 'tss', fast_mode=True, verbose=False)

if compute_graf: # if we filter data we use nb201_features.csv, if we dont filter we use nb201_features_all
    if os.path.exists(f"./tss_features_{dataset}.pickle"):
        archs = pd.read_pickle(f"./tss_features_{dataset}.pickle")
    else:
        archs = generate_accs(api_nats, dataset=dataset)
        print(f'No. of generated archs: {archs.shape[0]}')
        archs.to_pickle(f"./tss_features_{dataset}.pickle")
else:
    if os.path.exists(f"./tss_features_{dataset}_all.pickle"):
        archs = pd.read_pickle(f"./tss_features_{dataset}_all.pickle")
    else:
        archs = generate_accs(api_nats, dataset=dataset, features_path='../../GRAF/zc_combine/data/nb201_features_all.csv')
        print(f'No. of generated archs: {archs.shape[0]}')
        archs.to_pickle(f"./tss_features_{dataset}_all.pickle")

In [5]:
api_wandb = wandb.Api()

log = None
results = None
for run_id, seed in run_dict[dataset].items():
    
    run = pd.DataFrame(api_wandb.run(run_id).scan_history())
    run.rename({'arch': 'net_str'}, axis=1, inplace=True)
    
    run = pd.merge(archs, run, on='net_str', how='inner')
    if compute_graf:
        run = run.loc[run['net'].notnull(), :]  # keep only nets with features
        
    print(f'No. of archs for seed {seed} after filtering: {run.shape[0]}.')
    
    for col in run.columns:
        if col not in ['net_str', 'net']:
            run[col] = run[col].astype(float)
    
    if 'jacov' in run.columns:        
        run['jacov'] = run['jacov'].fillna(run['jacov'].min()).astype(float)

    df_scores = get_scores(run.copy(), compute_graf=compute_graf, zero_cost_score_list=zero_cost_score_list)
    df_scores['dataset'] = dataset
    df_scores['seed'] = seed
    if results is None:
        results = df_scores
    else:
        results = pd.concat([results, df_scores], ignore_index=True)
print(f'Total number of records: {results.shape[0]}')

No. of archs for seed 1 after filtering: 15625.
Running vkdnw
Running vkdnw_dim
Running vkdnw_chisquare
Running az_nas
Running jacov
Running gradsign
Running zico
Running zen
Running grad_norm
Running naswot
Running synflow
Running snip
Running grasp
Running te_nas
Running flops
Running trainability
Running expressivity
Running progressivity
No. of archs for seed 2 after filtering: 15625.
Running vkdnw
Running vkdnw_dim
Running vkdnw_chisquare
Running az_nas
Running jacov
Running gradsign
Running zico
Running zen
Running grad_norm
Running naswot
Running synflow
Running snip
Running grasp
Running te_nas
Running flops
Running trainability
Running expressivity
Running progressivity
No. of archs for seed 3 after filtering: 15625.
Running vkdnw
Running vkdnw_dim
Running vkdnw_chisquare
Running az_nas
Running jacov
Running gradsign
Running zico
Running zen
Running grad_norm
Running naswot
Running synflow
Running snip
Running grasp
Running te_nas
Running flops
Running trainability
Running exp

In [6]:
log = None
for seed in results['seed'].unique():
    
    results_temp = results.loc[results['seed'] == seed, :].copy()
    for zero_cost_rank in [p for p in results_temp.columns if '_rank' in p]:
        results_temp[[zero_cost_rank]] = results_temp[[zero_cost_rank]].apply(lambda x: x.replace(-np.inf, x[x != -np.inf].min()))
        results_temp[[zero_cost_rank]] = results_temp[[zero_cost_rank]].apply(lambda x: x.replace(-np.nan, x[x != -np.inf].min()))
        results_temp[[zero_cost_rank]] = results_temp[[zero_cost_rank]].apply(lambda x: x.replace(np.inf, x[x != np.inf].max()))
        log_temp = pd.DataFrame(get_metrics(results_temp, pred_name=zero_cost_rank, show_plot=False, seed=seed))
        
        if log is None:
            log = log_temp.copy()
        else:
            log = pd.concat([log, log_temp.copy()], ignore_index=True)
            
        if seed == min(results['seed'].unique()):
            plot_stats(get_stats(results_temp, 'vkdnw_dim', target, zero_cost_rank), 'vkdnw_dim', target, zero_cost_rank, f'{dataset}_{str(compute_graf)}_{zero_cost_rank}')

log = log.groupby('pred_name', as_index=False).agg(['mean', 'std']).reset_index()
log['dataset'] = dataset
log['no_seeds'] = len(results['seed'].unique())
log['archs_filtered'] = compute_graf_str
log

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript back

pred_name   kendall            spearman             pearson  \
                              mean       std      mean       std      mean   
0            az_nas_rank  0.694068  0.000562  0.875833  0.000452  0.890407   
1      expressivity_rank  0.497320  0.000311  0.686709  0.000262  0.714804   
2             flops_rank  0.517203  0.000000  0.690739  0.000000  0.527361   
3         grad_norm_rank  0.321994  0.000549  0.441007  0.000609  0.072047   
4          gradsign_rank -0.203849  0.001145 -0.265334  0.001573  0.075728   
5             grasp_rank  0.332788  0.003401  0.470419  0.005066  0.000364   
6             jacov_rank  0.536557  0.000999  0.701224  0.001168  0.583584   
7            naswot_rank  0.582778  0.000095  0.768348  0.000150  0.813633   
8     progressivity_rank  0.465775  0.001809  0.628155  0.001766  0.731978   
9              snip_rank  0.402701  0.000380  0.539273  0.000580  0.142569   
10          synflow_rank  0.555346  0.000132  0.747074  0.000094  0.069203   
11           te_nas_rank  0.458982  0.001588  0.640823  0.001950  0.659928   
12     trainability_rank  0.427419  0.000662  0.603455  0.000658  0.422402   
13  vkdnw_chisquare_rank -0.049211  0.001688 -0.065956  0.001925 -0.396736   
14       vkdnw_comb_rank  0.722457  0.000430  0.892737  0.000241  0.899090   
15        vkdnw_dim_rank  0.574273  0.000000  0.716298  0.000000  0.653620   
16        vkdnw_exp_rank  0.699342  0.000704  0.883235  0.000586  0.867293   
17       vkdnw_prog_rank  0.709373  0.000522  0.889171  0.000356  0.866040   
18            vkdnw_rank  0.602488  0.000496  0.793171  0.000453  0.736832   
19      vkdnw_train_rank  0.654578  0.000722  0.839130  0.000635  0.839476   
20              zen_rank  0.250367  0.000065  0.344212  0.000139  0.453846   
21             zico_rank  0.593763  0.000562  0.786932  0.000414  0.746817   

                        ndcg50                     ndcg100  ...  ndcg1000  \
             std          mean           std          mean  ...       std   
0   9.830528e-04  3.351248e-01  2.438388e-02  3.778947e-01  ...  0.013555   
1   2.058046e-03  8.530644e-03  8.126767e-04  1.034142e-02  ...  0.004351   
2   0.000000e+00  1.201383e-01  0.000000e+00  1.633045e-01  ...  0.000000   
3   7.109490e-03  3.749862e-04  2.213798e-04  8.434647e-03  ...  0.002147   
4   2.497168e-04  5.259106e-09  4.987548e-09  1.837850e-08  ...  0.000003   
5   2.472315e-02  4.186621e-03  2.630362e-03  1.068401e-02  ...  0.003234   
6   9.569804e-03  7.209319e-02  2.506434e-02  8.992557e-02  ...  0.012082   
7   6.572904e-05  3.198674e-01  3.261148e-03  2.794723e-01  ...  0.003059   
8   2.797029e-03  8.629835e-02  5.556431e-03  6.872144e-02  ...  0.005489   
9   8.401529e-03  1.220818e-03  4.913652e-04  1.499050e-02  ...  0.002251   
10  5.248292e-04  2.620345e-01  1.980444e-02  3.104594e-01  ...  0.005281   
11  1.987277e-03  1.302350e-02  2.615871e-03  1.675880e-02  ...  0.007905   
12  7.491938e-10  2.599841e-02  7.982936e-03  3.795140e-02  ...  0.002948   
13  8.947795e-04  7.306869e-15  4.177254e-25  8.726633e-15  ...  0.000449   
14  1.675542e-04  3.310347e-01  2.803305e-02  3.719112e-01  ...  0.007283   
15  0.000000e+00  2.768194e-01  0.000000e+00  3.260993e-01  ...  0.000000   
16  4.466231e-04  3.619830e-01  2.607022e-02  4.076121e-01  ...  0.013013   
17  2.897019e-04  3.574514e-01  1.441438e-02  4.006960e-01  ...  0.005010   
18  3.787998e-04  2.981178e-01  2.043445e-02  3.679977e-01  ...  0.006452   
19  1.069831e-03  1.156216e-01  2.125832e-02  1.701892e-01  ...  0.011041   
20  1.520580e-04  3.315691e-03  1.585250e-04  2.694498e-03  ...  0.000799   
21  5.204797e-05  2.625693e-01  1.031410e-02  2.797526e-01  ...  0.002243   

    ndcg5000              acc_top            acc_top_true       \
        mean       std       mean        std         mean  std   
0   0.735250  0.006524  45.573333   0.651108    47.311111  0.0   
1   0.560347  0.001186  39.368889   1.000259    47.311111  0.0   
2   0.645753  0.000000  41.44

In [7]:
"""
for zero_cost_score in ['vkdnw']:
    for i in range(5):
        results_temp = results.loc[results['seed'] == results['seed'].unique().min(), :].copy().sample(n=3000)
        analyze_results(api_nats, results_temp, zero_cost_score, target)
        
df_top = None
for seed in range(1, 6):
    results_temp = results.loc[results['seed'] == seed, :].sample(n=3000, random_state=seed).copy()
    for pred in [p for p in results_temp.columns if '_rank' in p]:
        top_acc = results_temp.loc[results_temp[pred].idxmax(), target]
        if df_top is None:
            df_top = pd.DataFrame({'acc': top_acc, 'acc_max': results_temp[target].max(), 'pred': pred, 'seed': seed}, index=[0])
        else:
            df_top = pd.concat([df_top, pd.DataFrame({'acc': top_acc, 'acc_max': results_temp[target].max(), 'pred': pred, 'seed': seed}, index=[0])], ignore_index=True)
df_top.groupby('pred', as_index=False)[['acc', 'acc_max']].agg(['max', 'mean', 'std']).reset_index()
"""

"\nfor zero_cost_score in ['vkdnw']:\n    for i in range(5):\n        results_temp = results.loc[results['seed'] == results['seed'].unique().min(), :].copy().sample(n=3000)\n        analyze_results(api_nats, results_temp, zero_cost_score, target)\n        \ndf_top = None\nfor seed in range(1, 6):\n    results_temp = results.loc[results['seed'] == seed, :].sample(n=3000, random_state=seed).copy()\n    for pred in [p for p in results_temp.columns if '_rank' in p]:\n        top_acc = results_temp.loc[results_temp[pred].idxmax(), target]\n        if df_top is None:\n            df_top = pd.DataFrame({'acc': top_acc, 'acc_max': results_temp[target].max(), 'pred': pred, 'seed': seed}, index=[0])\n        else:\n            df_top = pd.concat([df_top, pd.DataFrame({'acc': top_acc, 'acc_max': results_temp[target].max(), 'pred': pred, 'seed': seed}, index=[0])], ignore_index=True)\ndf_top.groupby('pred', as_index=False)[['acc', 'acc_max']].agg(['max', 'mean', 'std']).reset_index()\n"

In [8]:
log_train = None
    
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

pred_lists = {
    'model_vkdnw': [p for p in results.columns if '_lambda_' in p] + ['vkdnw_entropy', 'vkdnw_dim', 'flops'], #['vkdnw_chisquare', 'vkdnw_dim', 'flops'],
    'model_vkdnw+zs': [p for p in results.columns if '_lambda_' in p] + ['vkdnw_dim', 'flops'] + ['expressivity', 'progressivity', 'trainability', 'jacov','gradsign', 'zico', 'zen', 'grad_norm', 'naswot', 'synflow', 'snip', 'grasp', 'ntk', 'linear_region'],
    'model_vkdnw+zs+graf': [p for p in results.columns if '_lambda_' in p] + ['vkdnw_dim', 'flops'] + ['expressivity', 'progressivity', 'trainability', 'jacov','gradsign', 'zico', 'zen', 'grad_norm', 'naswot', 'synflow', 'snip', 'grasp', 'ntk', 'linear_region'] + [p for p in results.columns if 'op_' in p] + [p for p in results.columns if 'node_' in p],
    'model_graf': [p for p in results.columns if 'op_' in p] + [p for p in results.columns if 'node_' in p]
}

for train_size in [1024]:
    for seed in results['seed'].unique():
        
        results_temp = results.loc[results['seed'] == seed, :].copy()
        results_temp = results_temp.apply(lambda x: x.replace(-np.inf, x[x != -np.inf].min()))
        results_temp = results_temp.apply(lambda x: x.replace(np.inf, x[x != np.inf].max()))
        for model_name, pred_list in pred_lists.items():
            
            train_df, test_df = train_test_split(results_temp, test_size=1 - (train_size / results_temp.shape[0]), random_state=seed)
            model = Pipeline([
                ('scaler', StandardScaler()),           # Step 1: Standardize features
                ('regressor', RandomForestRegressor(n_estimators=100))  # Step 2: Train RandomForestRegressor
            ])
            model.fit(train_df[pred_list], train_df[target])
            test_df['pred_' + model_name] = model.predict(test_df[pred_list])
            log_train_temp = pd.DataFrame(get_metrics(test_df, 'pred_' + model_name, show_plot=False, seed=seed))        
            log_train_temp['train_size'] = train_size
        
            if log_train is None:
                log_train = log_train_temp.copy()
            else:
                log_train = pd.concat([log_train, log_train_temp.copy()], ignore_index=True)
                
            if seed == min(results['seed'].unique()):
                plot_stats(get_stats(test_df, 'vkdnw_dim', target, 'pred_' + model_name), 'vkdnw_dim', target, model_name, f'{dataset}_{str(compute_graf)}_{model_name}_{train_size}')

log_train = log_train.groupby(['pred_name', 'train_size'], as_index=False).agg(['mean', 'std']).reset_index()
log_train['dataset'] = dataset
log_train['no_seeds'] = len(results['seed'].unique())
log_train['archs_filtered'] = compute_graf_str
log_train

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


pred_name train_size   kendall            spearman  \
                                            mean       std      mean   
0           pred_model_graf       1024  0.811533  0.004657  0.945991   
1          pred_model_vkdnw       1024  0.641838  0.006935  0.825789   
2       pred_model_vkdnw+zs       1024  0.863391  0.002816  0.970819   
3  pred_model_vkdnw+zs+graf       1024  0.874178  0.002763  0.975391   

              pearson              ndcg50            ...  ndcg1000  ndcg5000  \
        std      mean       std      mean       std  ...       std      mean   
0  0.002141  0.964364  0.001879  0.605161  0.093648  ...  0.016902  0.905132   
1  0.005660  0.872478  0.004343  0.231657  0.051021  ...  0.031329  0.748108   
2  0.001591  0.982965  0.001225  0.674136  0.086861  ...  0.021387  0.915327   
3  0.001452  0.985059  0.001156  0.668232  0.082144  ...  0.018517  0.922472   

               acc_top           acc_top_true                   dataset  \
        std       mean       std         mean       std                   
0  0.012164  45.943333  0.441840    47.282222  0.064598  ImageNet16-120   
1  0.015992  43.635556  1.880302    47.282222  0.064598  ImageNet16-120   
2  0.015766  46.481111  0.729383    47.282222  0.064598  ImageNet16-120   
3  0.013579  46.288889  0.650973    47.282222  0.064598  ImageNet16-120   

  no_seeds archs_filtered  
                           
0        5     unfiltered  
1        5     unfiltered  
2        5     unfiltered  
3        5     unfiltered  

[4 rows x 23 columns]

In [9]:
log_train

pred_name train_size   kendall            spearman  \
                                            mean       std      mean   
0           pred_model_graf       1024  0.811533  0.004657  0.945991   
1          pred_model_vkdnw       1024  0.641838  0.006935  0.825789   
2       pred_model_vkdnw+zs       1024  0.863391  0.002816  0.970819   
3  pred_model_vkdnw+zs+graf       1024  0.874178  0.002763  0.975391   

              pearson              ndcg50            ...  ndcg1000  ndcg5000  \
        std      mean       std      mean       std  ...       std      mean   
0  0.002141  0.964364  0.001879  0.605161  0.093648  ...  0.016902  0.905132   
1  0.005660  0.872478  0.004343  0.231657  0.051021  ...  0.031329  0.748108   
2  0.001591  0.982965  0.001225  0.674136  0.086861  ...  0.021387  0.915327   
3  0.001452  0.985059  0.001156  0.668232  0.082144  ...  0.018517  0.922472   

               acc_top           acc_top_true                   dataset  \
        std       mean       std         mean       std                   
0  0.012164  45.943333  0.441840    47.282222  0.064598  ImageNet16-120   
1  0.015992  43.635556  1.880302    47.282222  0.064598  ImageNet16-120   
2  0.015766  46.481111  0.729383    47.282222  0.064598  ImageNet16-120   
3  0.013579  46.288889  0.650973    47.282222  0.064598  ImageNet16-120   

  no_seeds archs_filtered  
                           
0        5     unfiltered  
1        5     unfiltered  
2        5     unfiltered  
3        5     unfiltered  

[4 rows x 23 columns]

In [10]:
log_save = pd.concat([log, log_train], ignore_index=True)
log_save.columns = ['_'.join(col) for col in log_save.columns]
log_save.to_csv(f'{dataset}_{compute_graf_str}_log.csv', index=False)
log_save

,pred_name_,kendall_mean,kendall_std,spearman_mean,spearman_std,pearson_mean,pearson_std,ndcg50_mean,ndcg50_std,ndcg100_mean,...,ndcg5000_mean,ndcg5000_std,acc_top_mean,acc_top_std,acc_top_true_mean,acc_top_true_std,dataset_,no_seeds_,archs_filtered_,train_size_
0,az_nas_rank,0.694068,0.000562,0.875833,0.000452,0.890407,9.830528e-04,3.351248e-01,2.438388e-02,3.778947e-01,...,0.735250,0.006524,45.573333,0.651108,47.311111,0.000000,ImageNet16-120,5,unfiltered,NaN
1,expressivity_rank,0.497320,0.000311,0.686709,0.000262,0.714804,2.058046e-03,8.530644e-03,8.126767e-04,1.034142e-02,...,0.560347,0.001186,39.368889,1.000259,47.311111,0.000000,ImageNet16-120,5,unfiltered,NaN
2,flops_rank,0.517203,0.000000,0.690739,0.000000,0.527361,0.000000e+00,1.201383e-01,0.000000e+00,1.633045e-01,...,0.645753,0.000000,41.444444,0.000000,47.311111,0.000000,ImageNet16-120,5,unfiltered,NaN
3,grad_norm_rank,0.321994,0.000549,0.441007,0.000609,0.072047,7.109490e-03,3.749862e-04,2.213798e-04,8.434647e-03,...,0.425472,0.000941,18.450000,10.127342,47.311111,0.000000,ImageNet16-120,5,unfiltered,NaN
4,gradsign_rank,-0.203849,0.001145,-0.265334,0.001573,0.075728,2.497168e-04,5.259106e-09,4.987548e-09,1.837850e-08,...,0.008764,0.000236,15.115556,3.593391,47.311111,0.000000,ImageNet16-120,5,unfiltered,NaN
5,grasp_rank,0.332788,0.003401,0.470419,0.005066,0.000364,2.472315e-02,4.186621e-03,2.630362e-03,1.068401e-02,...,0.418082,0.003957,12.051111,10.259637,47.311111,0.000000,ImageNet16-120,5,unfiltered,NaN
6,jacov_rank,0.536557,0.000999,0.701224,0.001168,0.583584,9.569804e-03,7.209319e-02,2.506434e-02,8.992557e-02,...,0.597233,0.005543,36.297778,4.744361,47.311111,0.000000,ImageNet16-120,5,unfiltered,NaN
7,naswot_rank,0.582778,0.000095,0.768348,0.000150,0.813633,6.572904e-05,3.198674e-01,3.261148e-03,2.794723e-01,...,0.750660,0.001246,37.108889,3.647275,47.311111,0.000000,ImageNet16-120,5,unfiltered,NaN
8,progressivity_rank,0.465775,0.001809,0.628155,0.001766,0.731978,2.797029e-03,8.629835e-02,5.556431e-03,6.872144e-02,...,0.457236,0.006375,41.868889,1.122570,47.311111,0.000000,ImageNet16-120,5,unfiltered,NaN
9,snip_rank,0.402701,0.000380,0.539273,0.000580,0.142569,8.401529e-03,1.220818e-03,4.913652e-04,1.499050e-02,...,0.433320,0.000725,0.833333,0.000000,47.311111,0.000000,ImageNet16-120,5,unfiltered,NaN
